[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elsamuel27/snapthon/blob/main/Snapthon.ipynb)

In [ ]:
#@title # Paso 1: Rellena el formulario a continuación y ejecuta la celda

mult = "Video" #@param ["Video", "Playlist"]
link = "" #@param {type: "string"}
ft = "Audio" #@param ["Video", "Audio"]
format = "mp3" #@param ["mp3", "mp4"] {allow-input: true}
route = "/content/Descargados"
purification = False #@param {type: "boolean"}
#@markdown La purificación es un proceso que hace que la música sea más propensa a ser compatible con coches, casets, etc, pero aumenta mucho el tiempo de descarga. Actívala marcando la casilla de arriba. No tiene ningún efecto al momento de descargar vídeo, por lo que se desactivará automáticamente

if ft == "Audio" and purification:
    purification = False

# Paso 2: Ejecuta las celdas ocultas y espera

In [ ]:
#@title Actualizaciones

!pip install -U pip
!sudo apt update -y

In [ ]:
#@title Instalaciones

!pip install pytube
!pip install pydub
!sudo apt install zip -y

In [ ]:
#@title Importaciones

import os
from pytube import YouTube
from pytube import Playlist
try:
    from google.colab import files
except:
    print("Uso fuera de Google Colab activado")
if purification:
    from pydub import AudioSegment

Una desventaja de que los nombres de los archivos descargados sean los títulos de los vídeos, es que pueden contener caracteres incompatibles con el sistema de archivos. Una solución sencilla que se me ha ocurrido es la siguiente: 

- Creamos una lista con todos los caracteres no permitidos por Windows (Pues Linux admite unos cuántos más y para curar en salud mejor tener en cuenta más caracteres)
- En vez de usar el título directamente, lo almacenamos en otra variable, y acto seguido usamos el método `.replace()` de las cadenas de caracteres para reemplazar los caracteres no permitidos por nada, ergo, borrarlos

In [ ]:
#@title Descargas en el entorno

characters = ["\\", "/", ":", "*", "?", "\"", "<", ">", "|"]

if mult == "Video":
    yt = YouTube(link)
    title = yt.title
    for c in characters:
      title = title.replace(c, "")
    file = f"{title}.{format}"

    if ft == "Video":
        yd = yt.streams.get_highest_resolution()
        yd.download(route, file)
        print("¡Vídeo descargado! Hasta luego :D")
    elif ft == "Audio":
        yd = yt.streams.get_audio_only()
        yd.download(route, file)
        print("¡Vídeo descargado! Hasta luego :D")
elif mult == "Playlist":
    pl = Playlist(link)
    if ft == "Video":
        for video in pl.videos:
            title = video.title
            for c in characters:
              title = title.replace(c, "")
            file = f"{title}.{format}"
            try:
                yd = video.streams.get_highest_resolution()
                yd.download(route, file)
                print("+1 vídeo en la carpeta")
            except:
                print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")
    elif ft == "Audio":
        for video in pl.videos:
            title = video.title
            for c in characters:
                title = title.replace(c, "")
            try:
                yd = video.streams.get_audio_only()
                yd.download(route, f"{title}.{format}")
                print("+1 vídeo en la carpeta")
            except:
                print("No se puede acceder a los datos de este vídeo. Saltamos al siguiente")

In [ ]:
#@title Purificación (Si se ha activado)
if purification:
    input_folder = "/content/Descargados"
    output_folder = "/content/PDescargados"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, file_name)
        output_file_path = os.path.join(output_folder, file_name.rsplit('.', 1)[0] + '.mp3')
        sound = AudioSegment.from_file(input_file_path)
        sound.export(output_file_path, format='mp3')

# Paso 3: Elige la opción que más te convenga

Recomendada la primera opción para listas con muchos vídeos, la segunda para unos pocos o directamente uno

In [ ]:
#@title Archivo ZIP

if purification:
    os.system("zip -r /content/bySnapthon.zip PDescargados")
    os.removedirs("rm -r PDescargados")
else:
    os.system("zip -r /content/bySnapthon.zip Descargados")
    os.system("rm -r Descargados")
try:
    files.download("/content/bySnapthon.zip")
except:
    print("Uso fuera de Google Colab detectado.")

In [ ]:
#@title Archivo/s suelto/s

try:
  if purification:
    for video in os.listdir("/content/PDescargados"):
      files.download(f"/content/Descargados/{video}")
  else: 
    for video in os.listdir("/content/Descargados"):
      files.download(f"/content/Descargados/{video}")
except:
  print("Uso fuera de Google Colab detectado")

# Paso 4: Disfruta el contenido descargado